# Review NLP Helpfulness Prediction PretrainedModels (Digital Music)

## Creating Spark Session & Importing All Necessary Libraries

In [1]:
import findspark
findspark.init()

In [2]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql import functions as F

import sparknlp

# Start SparkSession with Spark NLP
# start() functions has 5 parameters: gpu, spark23, spark24, spark32, and memory
# sparknlp.start(gpu=True) will start the session with GPU support
# sparknlp.start(spark23=True) is when you have Apache Spark 2.3.x installed
# sparknlp.start(spark24=True) is when you have Apache Spark 2.4.x installed
# sparknlp.start(spark32=True) is when you have Apache Spark 3.2.x installed
# sparknlp.start(memory="16G") to change the default driver memory in SparkSession
spark = sparknlp.start(gpu = True, spark32= True)

### Reading in Data

In [3]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv(r"C:\Users\kenne\OneDrive\Desktop\UChicago\Python\Final Project Big Data Amazon Review\Amazon Review Kaggle Data\amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv",inferSchema=True,header=True, sep='\t' )

## Code Cleaning

In [4]:
df.show(1, vertical = True, truncate = False)

-RECORD 0------------------------------------------
 marketplace       | US                            
 customer_id       | 10293599                      
 review_id         | R14LVB34Z2Z53I                
 product_id        | B000YMOQZY                    
 product_parent    | 262935067                     
 product_title     | Knockin' On Heaven's Door     
 product_category  | Digital_Music_Purchase        
 star_rating       | 5                             
 helpful_votes     | 0                             
 total_votes       | 0                             
 vine              | N                             
 verified_purchase | Y                             
 review_headline   | favorites                     
 review_body       | Great  rendition. Great  song 
 review_date       | 2015-08-31                    
only showing top 1 row



In [5]:
df = df.select('product_id', 'star_rating', 'product_category', 'review_headline', 'review_body', 'helpful_votes', 'total_votes')

In [6]:
df.show(1, vertical = True, truncate = False)

-RECORD 0-----------------------------------------
 product_id       | B000YMOQZY                    
 star_rating      | 5                             
 product_category | Digital_Music_Purchase        
 review_headline  | favorites                     
 review_body      | Great  rendition. Great  song 
 helpful_votes    | 0                             
 total_votes      | 0                             
only showing top 1 row



In [7]:
df.dropna().count()

1688522

In [8]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|product_id|star_rating|product_category|review_headline|review_body|helpful_votes|total_votes|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|         0|          3|               3|            113|        254|            3|          3|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+



In [9]:
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.count()
df2.show(5, vertical = True)

-RECORD 0----------------
 product_id       | 2    
 star_rating      | 3    
 product_category | 3    
 review_headline  | 248  
 review_body      | 2061 
 helpful_votes    | 3    
 total_votes      | 3    



In [10]:
df = df.filter(col('total_votes') > 10)
df.count()

18631

In [11]:
df = df.fillna("", "review_body")
df = df.fillna("", "review_headline")

In [12]:
df = df.withColumn('review_text', F.concat('review_headline', F.lit(" "), 'review_body'))
df.show(1, vertical = True, truncate = False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 product_id       | B00ZQKVD4Q                                                                                                                                                                                                                                                                           

In [13]:
df.filter((col("review_text").isNull()) | ( col("review_text") == "")).show(1, vertical = True, truncate = False)

(0 rows)



In [14]:
df.select([count(when((col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns]).show(5, vertical = True)

-RECORD 0---------------
 product_id       | 0   
 star_rating      | 0   
 product_category | 0   
 review_headline  | 0   
 review_body      | 2   
 helpful_votes    | 0   
 total_votes      | 0   
 review_text      | 0   



In [15]:
df = df.withColumn('helpful_ratio', F.col('helpful_votes') / F.col('total_votes'))
df.show(1, vertical = True, truncate = False)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 product_id       | B00ZQKVD4Q                                                                                                                                                                                                                                                                           

In [16]:
df.filter(col('helpful_ratio') < 0).count()

0

In [17]:
df = df.withColumn('helpful', when(col("helpful_ratio") < 0.5, 0).otherwise(1))

In [18]:
df.show(5, vertical = True)

-RECORD 0--------------------------------
 product_id       | B00ZQKVD4Q           
 star_rating      | 5                    
 product_category | Digital_Music_Pur... 
 review_headline  | Excellent Jazz album 
 review_body      | Excellent album. ... 
 helpful_votes    | 30                   
 total_votes      | 30                   
 review_text      | Excellent Jazz al... 
 helpful_ratio    | 1.0                  
 helpful          | 1                    
-RECORD 1--------------------------------
 product_id       | B011JBCME0           
 star_rating      | 5                    
 product_category | Digital_Music_Pur... 
 review_headline  | Addictive, Fun Tune  
 review_body      | At 42, I might ra... 
 helpful_votes    | 9                    
 total_votes      | 12                   
 review_text      | Addictive, Fun Tu... 
 helpful_ratio    | 0.75                 
 helpful          | 1                    
-RECORD 2--------------------------------
 product_id       | B014DI9R94    

## USE Pipeline

In [19]:
#USE (Universal Sentence Encoder) Sentence Embedding
document = DocumentAssembler()\
    .setInputCol("review_text")\
    .setOutputCol("document")
    
embeddingsSentence = UniversalSentenceEncoder.load('tfhub_use_en_2.4.0_2.4_1587136330099') \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("helpful")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddingsSentence,
        classifierdl
    ])

In [20]:
%%time
pipeline2 = use_clf_pipeline.fit(df)

Wall time: 23.6 s


In [21]:
%%time
final = pipeline2.transform(df)

Wall time: 51.5 ms


In [22]:
metrics = final.select('helpful','product_id',"prediction.result")

## USE Results

In [23]:
%%time
from sklearn.metrics import classification_report, accuracy_score
metrics_final = metrics.toPandas()
metrics_final['result'] = metrics_final['result'].apply(lambda x: x[0])
metrics_final['result'] = metrics_final['result'].astype('int')

print(classification_report(metrics_final.helpful, metrics_final.result))
print(accuracy_score(metrics_final.helpful, metrics_final.result))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6135
           1       0.67      1.00      0.80     12496

    accuracy                           0.67     18631
   macro avg       0.34      0.50      0.40     18631
weighted avg       0.45      0.67      0.54     18631

0.6707101068112286
Wall time: 4.73 s


C:\Users\kenne\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BERT_uncased Embedding Pipeline

In [24]:
#BERT Sentence Embedding
document = DocumentAssembler()\
    .setInputCol("review_text")\
    .setOutputCol("document")
    
bert_cmlm = BertSentenceEmbeddings.load('sent_bert_base_uncased_en_2.6.0_2.4_1598346203624')\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("helpful")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddingsSentence,
        classifierdl
    ])

In [25]:
%%time
pipeline2 = use_clf_pipeline.fit(df)

Wall time: 14.2 s


In [26]:
%%time
final = pipeline2.transform(df)

Wall time: 39.5 ms


In [27]:
metrics = final.select('helpful','product_id',"prediction.result")

## BERT_uncased Embedding Results

In [28]:
%%time
from sklearn.metrics import classification_report, accuracy_score
metrics_final = metrics.toPandas()
metrics_final['result'] = metrics_final['result'].apply(lambda x: x[0])
metrics_final['result'] = metrics_final['result'].astype('int')

print(classification_report(metrics_final.helpful, metrics_final.result))
print(accuracy_score(metrics_final.helpful, metrics_final.result))

              precision    recall  f1-score   support

           0       0.77      0.67      0.72      6135
           1       0.85      0.90      0.87     12496

    accuracy                           0.83     18631
   macro avg       0.81      0.79      0.80     18631
weighted avg       0.82      0.83      0.82     18631

0.8261499651119103
Wall time: 4.83 s
